In [32]:
import pandas as pd
import os
import ast
data_path = '/home/evangelos/src/disaster-impact/data_mid/data_standardised/'


standardised_dfs = {
    "glide": pd.read_csv(os.path.join(data_path, 'glide_standardised.csv')),
    "gdacs": pd.read_csv(os.path.join(data_path, 'gdacs_standardised.csv')),
    "disaster_charter": pd.read_csv(os.path.join(data_path, 'disaster_charter_standardised.csv')),
    "emdat": pd.read_csv(os.path.join(data_path, 'emdat_standardised.csv')),
    "idmc": pd.read_csv(os.path.join(data_path, 'idmc_standardised.csv')),
    "cerf": pd.read_csv(os.path.join(data_path, 'cerf_standardised.csv')),
    "ifrc": pd.read_csv(os.path.join(data_path, 'ifrc_standardised.csv'))
}

# for name, df in standardised_dfs.items():
#     print(f"Dataset: {name}, Shape: {df.shape}")
#     if not df.empty:
#         display(df.iloc[0])
#     else:
#         print("The DataFrame is empty.")


def prefix_event_ids(value, prefix):
    if pd.isna(value):
        return None
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        try:
            parsed = ast.literal_eval(value)
            if not isinstance(parsed, list):
                parsed = [parsed]
            return [f"{prefix}_{item}" for item in parsed]
        except:
            return f"{prefix}_{value}"
    else:
        if isinstance(value, list):
            return [f"{prefix}_{item}" for item in value]
        else:
            return f"{prefix}_{value}"

for name, df in standardised_dfs.items():
    if "Event_ID" in df.columns:
        df["Event_ID"] = df["Event_ID"].apply(lambda x: prefix_event_ids(x, name))


In [33]:
all_data = pd.concat(standardised_dfs.values(), ignore_index=True)

/tmp/ipykernel_509767/201937051.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat(standardised_dfs.values(), ignore_index=True)


In [36]:
import hashlib

group_key = ['Event_Type', 'Country', 'Date']

def consolidate_group(group):
    consolidated_row = {}
    event_ids = sorted(set(group['Event_ID'].dropna().astype(str).tolist()))
    consolidated_row["Event_ID"] = event_ids

    unique_str = "|".join(event_ids)
    disaster_impact_id = "DI_" + hashlib.md5(unique_str.encode("utf-8")).hexdigest()
    consolidated_row["Disaster_Impact_ID"] = disaster_impact_id

    for column in group.columns:
        if column in group_key or column == "Event_ID" or column == "Disaster_Impact_ID":
            if column == "Disaster_Impact_ID":
                continue
            consolidated_row[column] = sorted(set(group[column].dropna().astype(str).tolist()))
        else:
            values = group[column].dropna().tolist()
            if values:
                if all(isinstance(val, list) for val in values):
                    flat_values = [item for sublist in values for item in sublist]
                    consolidated_row[column] = sorted(set(map(str, flat_values)))
                else:
                    consolidated_row[column] = sorted(set(map(str, values)))
            else:
                consolidated_row[column] = None

    return consolidated_row



In [37]:
unified_rows = []
for _, group in all_data.groupby(group_key):
    unified_rows.append(consolidate_group(group))

unified_df = pd.DataFrame(unified_rows)

cols = ['Disaster_Impact_ID', 'Event_ID'] + [c for c in unified_df.columns if c not in ('Disaster_Impact_ID','Event_ID')]
unified_df = unified_df[cols]

print(f"Unified DataFrame shape: {unified_df.shape}")
display(unified_df.head())

Unified DataFrame shape: (66886, 29)


,Disaster_Impact_ID,Event_ID,Source_Event_IDs,Event_Name,Event_Type,Country,Country_Code,Location,Latitude,Longitude,...,Financial_Loss,Alert_Level,Source,Comments,External_Links,AID_Contribution,Admin_Units,External_IDs,Approval_Date,Disbursement_Date
0,DI_38b6c39da0f00ce35fb11b327947c987,[glide_e0c9f247a7808cc69f6652e56d051d99],[['AC-2000-000005-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Fuging']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['(Road)']""]]",[[]],None,[[]],[[]],None,None
1,DI_55a537b19ff1fb6709ace74673fe1687,[glide_b58780e7c552092bf2392bc2d1b0b8be],[['AC-2000-000006-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Xiangtan (Hunan Province)']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['{Hotel} (Misc:Fire)']""]]",[[]],None,[[]],[[]],None,None
2,DI_3349612a14a4481e3b7f3c16dec96300,[glide_649016f3fb106db945bfcc081e4c3bab],[['AC-2000-000034-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Jiangsu Province']],[[32.061707]],[[118.763232]],...,None,[[]],[['[]']],"[[""['{Coal Mine} (Ind:Collapse)']""]]",[[]],None,[[]],[[]],None,None
3,DI_f95438b2942c1a1ded4297e3f5dcddbb,[glide_8f1649c0fc7a534bf0d99a381cd57524],[['AC-2000-000093-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Guigang (Guangxi Province)']],[[23.111531]],[[109.598927]],...,None,[[]],[['[]']],"[[""['(Misc:Fire)']""]]",[[]],None,[[]],[[]],None,None
4,DI_2fc9b60a9b7cc8ce4e7fc276d7b3eb5a,[glide_cf14cab0de23931d096296308952c1d1],[['AC-2000-000026-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Shunde']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['(Road)']""]]",[[]],None,[[]],[[]],None,None


In [38]:
os.makedirs('/home/evangelos/src/disaster-impact/data_out/data_unified/', exist_ok=True)

unified_df.to_csv('/home/evangelos/src/disaster-impact/data_out/data_unified/unified_data.csv', index=False)

In [40]:
analysis_df = unified_df.copy()
display(unified_df.iloc[28412])
display(analysis_df.head())


Disaster_Impact_ID                   DI_81e67d07f47f3357f4d9258ab88a0991
Event_ID               [gdacs_00af0e3dabd7e9d76971e104aac4bf6b, glide...
Source_Event_IDs       [['EQ-2004-000093-JPN'], ['[7889]', '[7856]', ...
Event_Name             [["['Earthquake in Japan']"], ["['Earthquake']"]]
Event_Type                                                      [['EQ']]
Country                                                      [['Japan']]
Country_Code                                               [["['JPN']"]]
Location               [['Near South Coast of Western Honshu'], ['["[...
Latitude               [['[33.05]', '[33.24]', '[33.35]', '[33.12]', ...
Longitude              [['[137.16]', '[136.79]', '[137.02]', '[136.87...
Date                                                        [2004-09-05]
Year                   [2004, [2004, 2004, 2004, 2004, 2004, 2004, 20...
Month                                                                [9]
Day                                                

,Disaster_Impact_ID,Event_ID,Source_Event_IDs,Event_Name,Event_Type,Country,Country_Code,Location,Latitude,Longitude,...,Financial_Loss,Alert_Level,Source,Comments,External_Links,AID_Contribution,Admin_Units,External_IDs,Approval_Date,Disbursement_Date
0,DI_38b6c39da0f00ce35fb11b327947c987,[glide_e0c9f247a7808cc69f6652e56d051d99],[['AC-2000-000005-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Fuging']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['(Road)']""]]",[[]],None,[[]],[[]],None,None
1,DI_55a537b19ff1fb6709ace74673fe1687,[glide_b58780e7c552092bf2392bc2d1b0b8be],[['AC-2000-000006-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Xiangtan (Hunan Province)']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['{Hotel} (Misc:Fire)']""]]",[[]],None,[[]],[[]],None,None
2,DI_3349612a14a4481e3b7f3c16dec96300,[glide_649016f3fb106db945bfcc081e4c3bab],[['AC-2000-000034-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Jiangsu Province']],[[32.061707]],[[118.763232]],...,None,[[]],[['[]']],"[[""['{Coal Mine} (Ind:Collapse)']""]]",[[]],None,[[]],[[]],None,None
3,DI_f95438b2942c1a1ded4297e3f5dcddbb,[glide_8f1649c0fc7a534bf0d99a381cd57524],[['AC-2000-000093-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Guigang (Guangxi Province)']],[[23.111531]],[[109.598927]],...,None,[[]],[['[]']],"[[""['(Misc:Fire)']""]]",[[]],None,[[]],[[]],None,None
4,DI_2fc9b60a9b7cc8ce4e7fc276d7b3eb5a,[glide_cf14cab0de23931d096296308952c1d1],[['AC-2000-000026-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Shunde']],[[19.0785907]],[[-98.2598043]],...,None,[[]],[['[]']],"[[""['(Road)']""]]",[[]],None,[[]],[[]],None,None


In [41]:
def has_prefix(id_list, prefix):
    if isinstance(id_list, list):
        return any(str(item).startswith(prefix + "_") for item in id_list)
    return False

analysis_df['gdacs'] = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'gdacs'))
analysis_df['glide'] = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'glide'))
analysis_df['cerf']  = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'cerf'))
analysis_df['charter'] = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'disaster_charter'))
analysis_df['emdat'] = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'emdat'))
analysis_df['idmc']  = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'idmc'))
analysis_df['dref']  = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'dref'))
analysis_df['ifrc']  = analysis_df['Event_ID'].apply(lambda lst: has_prefix(lst, 'ifrc'))

analysis_df['nb_sources'] = analysis_df[[
    'gdacs','glide','cerf','charter','emdat','idmc','dref','ifrc'
]].sum(axis=1)

display(analysis_df.head(10))


,Disaster_Impact_ID,Event_ID,Source_Event_IDs,Event_Name,Event_Type,Country,Country_Code,Location,Latitude,Longitude,...,Disbursement_Date,gdacs,glide,cerf,charter,emdat,idmc,dref,ifrc,nb_sources
0,DI_38b6c39da0f00ce35fb11b327947c987,[glide_e0c9f247a7808cc69f6652e56d051d99],[['AC-2000-000005-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Fuging']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1
1,DI_55a537b19ff1fb6709ace74673fe1687,[glide_b58780e7c552092bf2392bc2d1b0b8be],[['AC-2000-000006-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Xiangtan (Hunan Province)']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1
2,DI_3349612a14a4481e3b7f3c16dec96300,[glide_649016f3fb106db945bfcc081e4c3bab],[['AC-2000-000034-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Jiangsu Province']],[[32.061707]],[[118.763232]],...,None,False,True,False,False,False,False,False,False,1
3,DI_f95438b2942c1a1ded4297e3f5dcddbb,[glide_8f1649c0fc7a534bf0d99a381cd57524],[['AC-2000-000093-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Guigang (Guangxi Province)']],[[23.111531]],[[109.598927]],...,None,False,True,False,False,False,False,False,False,1
4,DI_2fc9b60a9b7cc8ce4e7fc276d7b3eb5a,[glide_cf14cab0de23931d096296308952c1d1],[['AC-2000-000026-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Shunde']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1
5,DI_6498c5e34d7c9dfe15cd17006390158e,[glide_ce1edc8b9e6333f7cddb748384a18ec3],[['AC-2000-000102-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Jilin Province']],[[43.896082]],[[125.326065]],...,None,False,True,False,False,False,False,False,False,1
6,DI_71f32a116fd5821258b52d4c3dde64e2,[glide_43f91c8e196e296cf7abf1d143573164],[['AC-2000-000114-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Dacheng County (Hebei Province)']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1
7,DI_39f897a8e8fe401272f2b151ecea6836,[glide_128f5892698e7533df4fb30445c9cdd8],[['AC-2000-000086-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Guangxi Province']],[[22.815478]],[[108.327546]],...,None,False,True,False,False,False,False,False,False,1
8,DI_79024f6e980a24e555401271019d9e1a,[glide_206d13add805d874b1f685819ee70b7d],[['AC-2000-000169-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Pingxiang (Jiangxi province)']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1
9,DI_77af5104e57ca5e71e4d5acd8adcd6c9,[glide_14b7e5276165e87de1226bddcc2b8d41],[['AC-2000-000154-CHN']],"[[""['Tech. Disaster']""]]",[['AC']],"[[""China, People's Republic""]]","[[""['CHN']""]]",[['Longjiang (Guangdong Province)']],[[19.0785907]],[[-98.2598043]],...,None,False,True,False,False,False,False,False,False,1


In [42]:
source_list = ['gdacs','glide','cerf','charter','emdat','idmc','dref','ifrc']
no_match_list = []
match_list = []

for source in source_list:
    single_source_mask = (analysis_df[source]) & (analysis_df['nb_sources'] == 1)
    multi_source_mask  = (analysis_df[source]) & (analysis_df['nb_sources'] > 1)

    x = single_source_mask.sum()  # number of rows that belong ONLY to this source
    y = multi_source_mask.sum()   # number of rows that belong to this and at least 1 more source

    no_match_list.append(x)
    match_list.append(y)

plot_df = pd.DataFrame()
plot_df['source'] = source_list
plot_df['no_match'] = no_match_list
plot_df['match'] = match_list
plot_df['total'] = plot_df['no_match'] + plot_df['match']
plot_df['score'] = plot_df['match'] / plot_df['total'].replace(0, None)

display(plot_df)


,source,no_match,match,total,score
0,gdacs,27134,324,27458,0.0118
1,glide,7133,324,7457,0.043449
2,cerf,0,0,0,NaN
3,charter,370,11,381,0.028871
4,emdat,10028,77,10105,0.00762
5,idmc,17805,0,17805,0.0
6,dref,0,0,0,NaN
7,ifrc,4015,69,4084,0.016895
